In [ ]:
%load_ext autoreload
%autoreload 2

import env

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

# --- 1. Загрузка конфигурации ---
service = env.get_gservice()

if service:
    df_sheet = env.read_df_from_spreadsheet(service, env.SHEET_ID, env.SHEET_NAME)
    print("Данные из Google Sheets загружены")
else:
    raise ConnectionError("Не удалось подключиться к Google API")

RS_TABLE = 'incent_opex_check_universal'
RS_SCHEMA = 'ma_data'

# Две отдельные проверки для previous_cr и historical_cr
CHECK_CONFIGS = ['01-incent.cr_prev', '01-incent.cr_hist']

# Хелпер для SQL списков
def to_sql_list(items):
    if not isinstance(items, list):
        items = [items] 
    if not items:
        return "()"
    
    formatted = []
    for x in items:
        if isinstance(x, str):
            formatted.append(f"'{x}'") 
        else:
            formatted.append(str(x))   
            
    return f"({', '.join(formatted)})"


# --- 2. Функции статистики ---

def calc_std_error(cr, n):
    return np.sqrt(np.divide(cr * (1 - cr), n, out=np.zeros_like(cr), where=n!=0))

def calc_reference_ci(cr, n, z):
    """Рассчитывает ширину доверительного интервала для reference-значения"""
    se = calc_std_error(cr, n)
    return z * se


# --- 3. Основная функция проверки ---

def run_check_for_window(target_cw, lag_weeks, level_rules_dict, reference_type, config):
    """
    reference_type: 'prev' или 'hist'
    """
    
    # Извлекаем параметры из config
    CONFIG_COUNTRIES = config['countries_sql']
    CONFIG_PARTNERS = config['partners_sql']
    N_SIGMAS = config['n_sigmas']
    MIN_INSTALLS = config['min_installs']
    MIN_USERS = config['min_users']
    CRITERIA = config['criteria']
    THRESHOLD_WARNING_PCT = config['threshold_warning']
    THRESHOLD_CRIT_PCT = config['threshold_crit']
    ALERT_CATEGORY = config['alert_category']
    CHECK_COUNTRIES = config['check_countries']
    
    # A. Формирование SQL условий
    conditions = []
    if 'exceptions' in level_rules_dict:
        for app_name, levels in level_rules_dict['exceptions'].items():
            levels_sql = to_sql_list(levels)
            conditions.append(f"(app = '{app_name}' AND level IN {levels_sql})")
        excluded_apps = list(level_rules_dict['exceptions'].keys())
    else:
        excluded_apps = []

    default_levels_sql = to_sql_list(level_rules_dict['default'])
    
    if excluded_apps:
        excl_apps_sql = to_sql_list(excluded_apps)
        default_cond = f"(app NOT IN {excl_apps_sql} AND level IN {default_levels_sql})"
    else:
        default_cond = f"(level IN {default_levels_sql})"
    
    conditions.append(default_cond)
    level_filter_sql = " AND (" + " OR ".join(conditions) + ")"
    
    # B. Расчет дат
    today = datetime.now().date()
    last_full_sunday = today - timedelta(days=today.weekday() + 1)
    
    current_end = last_full_sunday - timedelta(weeks=lag_weeks - 1)
    current_start = current_end - timedelta(days=6)
    
    prev_end = current_start - timedelta(days=1)
    prev_start = prev_end - timedelta(days=6)
    
    history_end = current_start - timedelta(days=1)
    history_start = history_end - timedelta(weeks=4) + timedelta(days=1)

    print(f"\n--- Checking CW={target_cw} (Lag: {lag_weeks} weeks, Type: {reference_type}) ---")
    
    # C. SQL Запрос - извлекаем current и нужный reference
    if reference_type == 'prev':
        sql_query = f"""
        WITH raw_data AS (
            SELECT 
                partner_id, app, store, country, level, cw,
                cohort_date::DATE as cohort_date_clean, 
                unique_user_count, installs
            FROM ma_data.vinokurov_cr_data
            WHERE 
                partner_id IN {CONFIG_PARTNERS}
                AND country IN {CONFIG_COUNTRIES}
                AND cw = {target_cw}
                {level_filter_sql} 
                AND cohort_date::DATE >= '{prev_start}' 
                AND cohort_date::DATE <= '{current_end}'
        ),
        previous_stats AS (
            SELECT partner_id, app, store, country, level,
                SUM(unique_user_count) as ref_users, SUM(installs) as ref_installs
            FROM raw_data
            WHERE cohort_date_clean BETWEEN '{prev_start}' AND '{prev_end}'
            GROUP BY partner_id, app, store, country, level
        ),
        current_stats AS (
            SELECT partner_id, app, store, country, level,
                SUM(unique_user_count) as curr_users, SUM(installs) as curr_installs,
                MIN(cohort_date_clean) as cohort_date
            FROM raw_data
            WHERE cohort_date_clean BETWEEN '{current_start}' AND '{current_end}'
            GROUP BY partner_id, app, store, country, level
        )
        SELECT 
            c.partner_id, c.app, c.store, c.country, c.level, {target_cw} as cw, c.cohort_date,
            c.curr_installs, c.curr_users,
            p.ref_installs, p.ref_users,
            (c.curr_users::float / NULLIF(c.curr_installs, 0)) as current_cr,
            (p.ref_users::float / NULLIF(p.ref_installs, 0)) as reference_cr
        FROM current_stats c
        JOIN previous_stats p USING (partner_id, app, store, country, level)
        """
    else:  # reference_type == 'hist'
        sql_query = f"""
        WITH raw_data AS (
            SELECT 
                partner_id, app, store, country, level, cw,
                cohort_date::DATE as cohort_date_clean, 
                unique_user_count, installs
            FROM ma_data.vinokurov_cr_data
            WHERE 
                partner_id IN {CONFIG_PARTNERS}
                AND country IN {CONFIG_COUNTRIES}
                AND cw = {target_cw}
                {level_filter_sql} 
                AND cohort_date::DATE >= '{history_start}' 
                AND cohort_date::DATE <= '{current_end}'
        ),
        historical_stats AS (
            SELECT partner_id, app, store, country, level,
                SUM(unique_user_count) as ref_users, SUM(installs) as ref_installs
            FROM raw_data
            WHERE cohort_date_clean BETWEEN '{history_start}' AND '{history_end}'
            GROUP BY partner_id, app, store, country, level
        ),
        current_stats AS (
            SELECT partner_id, app, store, country, level,
                SUM(unique_user_count) as curr_users, SUM(installs) as curr_installs,
                MIN(cohort_date_clean) as cohort_date
            FROM raw_data
            WHERE cohort_date_clean BETWEEN '{current_start}' AND '{current_end}'
            GROUP BY partner_id, app, store, country, level
        )
        SELECT 
            c.partner_id, c.app, c.store, c.country, c.level, {target_cw} as cw, c.cohort_date,
            c.curr_installs, c.curr_users,
            h.ref_installs, h.ref_users,
            (c.curr_users::float / NULLIF(c.curr_installs, 0)) as current_cr,
            (h.ref_users::float / NULLIF(h.ref_installs, 0)) as reference_cr
        FROM current_stats c
        JOIN historical_stats h USING (partner_id, app, store, country, level)
        """
    
    df = env.execute_sql(sql_query)
    
    # Если данных нет
    df = df.fillna(0)
    if df.empty:
        print(f"  >> No data found for CW={target_cw}. Skipping.")
        return df

    print(f"  >> Data fetched: {len(df)} rows")

    # --- Подготовка данных ---
    
    numeric_raw_cols = ['partner_id', 'curr_installs', 'curr_users', 'ref_installs', 'ref_users']
    for col in numeric_raw_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # Агрегация ALL (по странам, но сохраняем partner_id)
    group_cols = ['partner_id', 'app', 'store', 'level', 'cw', 'cohort_date']
    sum_cols = ['curr_installs', 'curr_users', 'ref_installs', 'ref_users']
    
    df_all = df.groupby(group_cols, as_index=False)[sum_cols].sum()
    df_all['country'] = 'ALL'
    
    if CHECK_COUNTRIES:
        df = pd.concat([df, df_all], ignore_index=True)
    else:
        df = df_all
    
    # Пересчет CR
    df['current_cr'] = np.where(df['curr_installs'] > 0, df['curr_users'] / df['curr_installs'], 0.0)
    df['reference_cr'] = np.where(df['ref_installs'] > 0, df['ref_users'] / df['ref_installs'], 0.0)

    calc_cols = ['current_cr', 'reference_cr', 'curr_installs', 'ref_installs']
    for col in calc_cols:
        df[col] = df[col].astype(float)

    # --- ФИЛЬТРАЦИЯ (Thresholds) ---
    
    df = df[
        (df['curr_installs'] >= MIN_INSTALLS) & 
        (df['ref_installs'] >= MIN_INSTALLS) &
        (df['curr_users'] >= MIN_USERS) & 
        (df['ref_users'] >= MIN_USERS)
    ].copy()
    
    if df.empty:
        return df

    # --- Расчет CI для reference-значения ---
    
    df['reference_value_ci'] = calc_reference_ci(df['reference_cr'], df['ref_installs'], N_SIGMAS)

    # --- Расчет change_perc ---
    df['change_perc'] = np.where(
        df['reference_cr'] > 0,
        (df['current_cr'] - df['reference_cr']) / df['reference_cr'],
        0.0
    )

    # --- Логика алертов ---
    if CRITERIA == 'change':
        abs_change = np.abs(df['change_perc'].values)
        
        # Условие 1: превышен threshold изменения
        change_condition = abs_change >= THRESHOLD_WARNING_PCT
        
        # Условие 2: текущее значение НЕ в доверительном интервале
        ci_condition = (df['current_cr'] < df['reference_cr'] - df['reference_value_ci']) | \
                       (df['current_cr'] > df['reference_cr'] + df['reference_value_ci'])
        
        # ОБА условия должны быть True
        df['is_alert'] = change_condition & ci_condition
        df['is_critical'] = (THRESHOLD_CRIT_PCT > 0) & (abs_change >= THRESHOLD_CRIT_PCT) & ci_condition
    else:
        # CI-based: проверяем попадание current_cr в диапазон reference ± ci
        df['is_alert'] = (df['current_cr'] < df['reference_cr'] - df['reference_value_ci']) | \
                         (df['current_cr'] > df['reference_cr'] + df['reference_value_ci'])
        df['is_critical'] = False
    
    return df


# --- 4. ГЛАВНЫЙ ЦИКЛ ПО КОНФИГУРАЦИЯМ ---

LAG_MAP = {7: 2, 30: 5, 90: 14}

all_final_results = []

for ALERT_NAME in CHECK_CONFIGS:
    print(f"\n{'='*60}")
    print(f"Обработка конфигурации: {ALERT_NAME}")
    print('='*60)
    
    try:
        config_row = df_sheet[df_sheet['name'] == ALERT_NAME].iloc[0]
    except IndexError:
        print(f"  >> Алерт '{ALERT_NAME}' не найден в Google Sheet. Пропуск.")
        continue

    if config_row['active_flag'] != 'Enabled':
        print(f"  >> Алерт '{ALERT_NAME}' отключен. Пропуск.")
        continue
    
    # --- Парсинг параметров ---
    N_SIGMAS = abs(float(config_row['n_sigmas'])) 
    MIN_INSTALLS = int(config_row['threshold_installs'])
    MIN_USERS = int(config_row['threshold_conv'])
    ALERT_CATEGORY = config_row['metric_crit_category']

    # Критерий формирования алертов: 'ci' или 'change'
    _criteria = config_row.get('criteria', 'ci')
    CRITERIA = str(_criteria).strip().lower() if pd.notna(_criteria) else 'ci'

    if CRITERIA == 'change':
        _tw = config_row.get('threshold_warning', 0)
        _tc = config_row.get('threshold_crit', 0)
        THRESHOLD_WARNING_PCT = abs(float(_tw)) if pd.notna(_tw) else 0
        THRESHOLD_CRIT_PCT = abs(float(_tc)) if pd.notna(_tc) else 0
    else:
        CRITERIA = 'ci'
        THRESHOLD_WARNING_PCT = 0
        THRESHOLD_CRIT_PCT = 0

    try:
        # Загружаем JSON настроек
        params = json.loads(config_row['config_json'])

        CONFIG_COUNTRIES = to_sql_list(params['countries'])
        
        # Парсинг partners (dict {id: name})
        partners_dict = params['partners']
        partner_ids = [int(pid) for pid in partners_dict.keys()]
        CONFIG_PARTNERS = to_sql_list(partner_ids)
        
        CONFIG_RULES = params['cw']
        
        # Флаг проверки стран
        check_countries_val = params.get('check_countries', 'TRUE')
        CHECK_COUNTRIES = str(check_countries_val).upper() == 'TRUE'
        
    except json.JSONDecodeError as e:
        print(f"  >> Ошибка JSON в ячейке config_json: {e}")
        continue
    except KeyError as e:
        print(f"  >> В JSON отсутствует обязательный ключ: {e}")
        continue

    print(f"Настройки: Sigma={N_SIGMAS}, MinInstalls={MIN_INSTALLS}, MinUsers={MIN_USERS}")
    print(f"Partners: {list(partners_dict.values())}")
    print(f"Check Countries: {CHECK_COUNTRIES}")
    if CRITERIA == 'change':
        print(f"Критерий: CHANGE (warning={THRESHOLD_WARNING_PCT:.1%}, crit={THRESHOLD_CRIT_PCT:.1%})")
    else:
        print(f"Критерий: CI (n_sigmas={N_SIGMAS})")
    
    # Определение reference_type
    if 'prev' in ALERT_NAME.lower():
        reference_type = 'prev'
        metric_name = 'previous_cr'
    else:
        reference_type = 'hist'
        metric_name = 'historical_cr'
    
    # Собираем конфиг в dict для передачи в функцию
    config = {
        'countries_sql': CONFIG_COUNTRIES,
        'partners_sql': CONFIG_PARTNERS,
        'n_sigmas': N_SIGMAS,
        'min_installs': MIN_INSTALLS,
        'min_users': MIN_USERS,
        'criteria': CRITERIA,
        'threshold_warning': THRESHOLD_WARNING_PCT,
        'threshold_crit': THRESHOLD_CRIT_PCT,
        'alert_category': ALERT_CATEGORY,
        'check_countries': CHECK_COUNTRIES
    }
    
    # --- Запуск цикла проверок ---
    result_frames = []
    
    for cw_key_str, rules in CONFIG_RULES.items():
        cw = int(cw_key_str) 
        lag = LAG_MAP.get(cw, 5) 
        
        df_res = run_check_for_window(cw, lag, rules, reference_type, config)
        if not df_res.empty:
            result_frames.append(df_res)
    
    # --- Формирование отчета для текущей конфигурации ---
    
    if result_frames:
        full_report = pd.concat(result_frames, ignore_index=True)
        
        if not full_report.empty:
            # Переименование колонок в универсальный формат
            full_report['metric'] = metric_name
            
            full_report = full_report.rename(columns={
                'level': 'slice1',
                'cw': 'slice2',
                'current_cr': 'current_value',
                'reference_cr': 'reference_value'
            })
            
            # alert_category
            if CRITERIA == 'change':
                full_report['alert_category'] = full_report.apply(
                    lambda r: 'CRITICAL' if r['is_alert'] and r['is_critical']
                              else ('WARNING' if r['is_alert'] else None), axis=1)
            else:
                full_report['alert_category'] = full_report['is_alert'].apply(
                    lambda x: 'WARNING' if x else None)
            
            full_report = full_report.drop(columns=['is_critical'], errors='ignore')
            
            # Добавляем общие поля
            full_report['date'] = datetime.now()
            full_report['check_name'] = '01-incent.cr'
            full_report['metric_crit_category'] = ALERT_CATEGORY
            full_report['segment'] = None
            full_report['slice3'] = None
            full_report['slice4'] = None
            
            # Формируем app_short
            store_suffix_map = {'googleplay': '_gp', 'ios': '_as'}
            full_report['app_short'] = full_report.apply(
                lambda row: row['app'] + store_suffix_map.get(row['store'], ''), axis=1
            )
            
            all_final_results.append(full_report)
            
            alerts_count = full_report['is_alert'].sum()
            print(f"\n[{ALERT_CATEGORY.upper()}] Всего записей: {len(full_report)}, из них алертов: {alerts_count}")
        else:
            print("Нет данных после фильтрации.")
    else:
        print("Нет данных.")


# --- 5. Объединение и запись в БД ---

if all_final_results:
    all_results = pd.concat(all_final_results, ignore_index=True)
    
    # Сортировка
    all_results = all_results.sort_values(
        by=['cohort_date', 'check_name', 'partner_id', 'app_short'], 
        ascending=[False, True, True, True]
    )
    
    # Список колонок для записи в БД
    db_cols = [
        'date', 'check_name', 'metric',
        'partner_id', 'app_short', 'country', 'segment',
        'slice1', 'slice2', 'slice3', 'slice4',
        'cohort_date', 'metric_crit_category',
        'current_value', 'reference_value', 'reference_value_ci',
        'change_perc', 'is_alert', 'alert_category'
    ]
    
    df_to_write = all_results[db_cols].copy()
    
    if not df_to_write.empty:
        print(f"\n{'='*60}")
        print(f"Запись {len(df_to_write)} строк в Redshift...")
        env.insert_table_into_rs(df_to_write, RS_TABLE, RS_SCHEMA, 10000)
        print("Успешно записано.")
    
    # Фильтруем алерты для вывода в отчёт
    alerts_final = all_results[all_results['is_alert'] == True].copy()

    # Вывод таблицы в отчет Jupyter (только алерты)
    if not alerts_final.empty:
        print(f"\n{'='*60}")
        print("ИТОГОВАЯ ТАБЛИЦА АЛЕРТОВ")
        print('='*60)
        display_cols = [
            'date', 'check_name', 'metric',
            'partner_id', 'app_short', 'country', 'slice1', 'slice2',
            'metric_crit_category', 'alert_category',
            'current_value', 'reference_value', 'reference_value_ci',
            'change_perc', 'is_alert'
        ]
        styled_df = alerts_final[display_cols].style.hide(axis='index').format({
            'current_value': '{:.2%}',
            'reference_value': '{:.2%}',
            'reference_value_ci': '{:.2%}',
            'change_perc': '{:+.1%}'
        })
        display(styled_df)
    else:
        print("\n{'='*60}")
        print("Значимых изменений не найдено.")
else:
    print("\nНет данных для записи.")